In [ ]:
import numpy as np
import scipy.io as sio

import pandas as pd

import matplotlib.pyplot as plt
import matplotlib

In [ ]:
rootdir = '/raid1/chen423/serdp/archive/GRL2020/'

plotdir = rootdir + 'plots/'

indir = rootdir + 'data/HIST/moisture/full_ocean/'

## 1.  moisture budget data

In [ ]:
indata = np.zeros((17532, 4))

sindex = 0
eindex = 0

year = 2003
print(year)
for month in np.arange(10,13):
    infile = indir + 'ETratio.HIST.full_ocean.%d.%d.mat' % (year, month)
    tmpdata = sio.loadmat(infile)['array_ET'][0]
    eindex = eindex + tmpdata.shape[0]
    #print(year, month, sindex, eindex)
    indata[sindex:eindex, 0] = sio.loadmat(infile)['array_ET'][0]
    indata[sindex:eindex, 1] = sio.loadmat(infile)['array_uIVT'][0]
    indata[sindex:eindex, 2] = sio.loadmat(infile)['array_vIVT_bottom'][0]
    indata[sindex:eindex, 3] = sio.loadmat(infile)['array_vIVT_top'][0]
    sindex = eindex
    
for year in np.arange(2004,2015):
    print(year)
    for month in np.arange(1,13):
        infile = indir + 'ETratio.HIST.full_ocean.%d.%d.mat' % (year, month)
        tmpdata = sio.loadmat(infile)['array_ET'][0]
        eindex = eindex + tmpdata.shape[0]
        #print(year, month, sindex, eindex)
        indata[sindex:eindex, 0] = sio.loadmat(infile)['array_ET'][0]
        indata[sindex:eindex, 1] = sio.loadmat(infile)['array_uIVT'][0]
        indata[sindex:eindex, 2] = sio.loadmat(infile)['array_vIVT_bottom'][0]
        indata[sindex:eindex, 3] = sio.loadmat(infile)['array_vIVT_top'][0]
        sindex = eindex
        
year = 2015
print(year)
for month in np.arange(1,10):
    infile = indir + 'ETratio.HIST.full_ocean.%d.%d.mat' % (year, month)
    tmpdata = sio.loadmat(infile)['array_ET'][0]
    eindex = eindex + tmpdata.shape[0]
    #print(year, month, sindex, eindex)
    indata[sindex:eindex, 0] = sio.loadmat(infile)['array_ET'][0]
    indata[sindex:eindex, 1] = sio.loadmat(infile)['array_uIVT'][0]
    indata[sindex:eindex, 2] = sio.loadmat(infile)['array_vIVT_bottom'][0]
    indata[sindex:eindex, 3] = sio.loadmat(infile)['array_vIVT_top'][0]
    sindex = eindex

In [ ]:
def compute_histogram(ARcase, hmax=50):
    if ARcase=='AR':
        valid_index =  ((indata[:,0]+indata[:,1]+indata[:,2]-indata[:,3])>0) & ((indata[:,1]+indata[:,2]-indata[:,3])>0) & (AR6hr_index[0]==1)
    elif ARcase=='nAR':
        valid_index =  ((indata[:,0]+indata[:,1]+indata[:,2]-indata[:,3])>0) & ((indata[:,1]+indata[:,2]-indata[:,3])>0) & (AR6hr_index[0]==0)

    ratios = (indata[valid_index,0]/(indata[valid_index,0]+indata[valid_index,1]+indata[valid_index,2]-indata[valid_index,3]))*100

    mois_weighted_ratio = indata[valid_index,0].sum()/(indata[valid_index,1].sum() + indata[valid_index,2].sum() - indata[valid_index,3].sum())*100
    ratio_hist, hist_bin_edge = np.histogram(ratios, bins=np.arange(0, hmax+0.00001, 0.5))
    nbins = hist_bin_edge.shape[0]-1
    
    hist_bin_ct = (hist_bin_edge[0:nbins]+hist_bin_edge[1:(nbins+1)])/2
    
    return ratios, hist_bin_ct, ratio_hist, nbins, mois_weighted_ratio

In [ ]:
ARtag = 'abs'
flag_area = 1000   # minimum size of patches (over land and over ocean, both)
flag_USstate = 1  # whether to use US west coast 5 states along with land mask. 1 is to use, 0 is to skip
flag_post_adj = 1  # WRF further adjusted, or not (i.e., directly from modified NARR). 1 is further adjusted, 0 for raw

commonAR_thre = 1000

version_tag = 'AR%s_s%d_state%d_post%d_c%d' % (ARtag, flag_area, flag_USstate, flag_post_adj, commonAR_thre)
print(version_tag)

tmpfile = rootdir + 'data/intermediate_data/AR_stats_separate.%s.mat' % version_tag
AR6hr_index = sio.loadmat(tmpfile)['ARday_index']

In [ ]:
ratio_AR, hist_bin_ct, hist_AR, nbins, m_ratio_AR = compute_histogram('AR', hmax=60)
ratio_nAR, _, hist_nAR, _, m_ratio_nAR = compute_histogram('nAR', hmax=60)

## 2. IWV/IVT sensivitiy

In [ ]:
tmpfile = rootdir + 'data/intermediate_data/full_ocean_stats_separate.mat'
    
stats_SSTmean = sio.loadmat(tmpfile)['stats_SSTmean']
stats_IVT = sio.loadmat(tmpfile)['stats_IVT']
stats_IWV = sio.loadmat(tmpfile)['stats_IWV']
stats_IVTf = sio.loadmat(tmpfile)['stats_IVTf']
stats_IWVf = sio.loadmat(tmpfile)['stats_IWVf']

In [ ]:
tmpfile = rootdir + 'data/intermediate_data/AR_stats_separate.%s.mat' % version_tag
AR6hr_index = sio.loadmat(tmpfile)['ARday_index']
common_AR = (AR6hr_index.sum(axis=0))==2
common_nAR = (AR6hr_index.sum(axis=0))==0

In [ ]:
def compute_IWV_IVT_SST_pairs(ARcase, region='ocean'):
    # region: "ocean" or "all"
    if ARcase=='AR':
        valid_index = (common_AR==1)*(np.abs(stats_SSTmean[0]-stats_SSTmean[1])>=0.1)
    elif ARcase=='nAR':
        valid_index = (common_nAR==1)*(np.abs(stats_SSTmean[0]-stats_SSTmean[1])>=0.1)

    valid_SSTmean = stats_SSTmean[:,valid_index==1]
    
    if region=='ocean':
        valid_IVT = stats_IVT[:, valid_index==1]
        valid_IWV = stats_IWV[:, valid_index==1]
    elif region=='all':
        valid_IVT = stats_IVTf[:, valid_index==1]
        valid_IWV = stats_IWVf[:, valid_index==1]
        
    diff_SST = valid_SSTmean[1]-valid_SSTmean[0]
    diff_IVT = valid_IVT[1]/valid_IVT[0]-1
    diff_IWV = valid_IWV[1]/valid_IWV[0]-1
        
    
    print('There are %d valid 6-hr snaps with co-occurrence of ARs' % valid_index.sum())
    
    return diff_SST, diff_IVT, diff_IWV

In [ ]:
def compute_regr_coeffs(inx, iny):
    nt = iny.shape[0]
    x = inx
    
    k = ((x*(iny-iny.mean())).sum()) / ((x*(x-x.mean())).sum())
    b = iny.mean() - k * x.mean()
    
    return k, b

import scipy.stats.distributions as dist


def check_mean_zero_significance(indata, thre_p=0.1):
    n = indata.shape[0]
    se = indata.std()/np.sqrt(n)
    be = indata.mean()
    he = 0
    test_statistic = (be - he)/se
    
    pvalue = 2*dist.norm.cdf(-np.abs(test_statistic))
    
    if np.abs(pvalue)<thre_p:
        sig = 1
    else:
        sig = 0
    
    return sig


def barplot_double_set_sig(axis, in_x, indata, inlabels, incolors):
    patch_collection_for_legend = []
    for i in np.arange(nbins):
        threb = -1*xmax+i*xres
        threu = -1*xmax+(i+1)*xres
        loc_v = (threb+threu)/2
        index = (in_x>threb)*(in_x<=threu)
        if index.sum()>20:
            plotdata = [indata[0][index], indata[1][index]]
            sigs = [check_mean_zero_significance(plotdata[0]), check_mean_zero_significance(plotdata[1])]
            #print(sigs)
            #bplot1 = ax1.boxplot(plotdata, positions=np.arange(loc_v-1*para_a-1*para_b, loc_v+1*para_a+1*para_b+0.0000001, para_a+para_b), widths=para_a,
            #            showfliers=False, whis=[10,90], patch_artist=True)
            bplot1 = axis.boxplot(plotdata, positions=np.arange(loc_v-0.5*para_a-0.5*para_b, loc_v+0.5*para_a+0.5*para_b+0.0000001, para_a+para_b), widths=para_a,
                        showfliers=False, whis=[10,90], patch_artist=True)
            for patch, color, sig in zip(bplot1['boxes'], incolors, sigs):
                if sig==1:
                    patch.set_facecolor(color)
                    patch.set_edgecolor(color) 
                if sig==0:
                    patch.set_facecolor('none')
                    patch.set_edgecolor('black')
                if i==9: # 7 for abs
                    patch_collection_for_legend.append(patch)
            if i==9: # 7 for abs
                leg = axis.legend(patch_collection_for_legend,
                                 inlabels, ncol=1, loc='upper left',
                                 frameon=False, fontsize=10)
                for legobj in leg.legendHandles:
                    legobj.set_linewidth(0)

In [ ]:
diff_SST_AR, diff_IVT_AR, diff_IWV_AR = compute_IWV_IVT_SST_pairs('AR', region='ocean')
diff_SST_nAR, diff_IVT_nAR, diff_IWV_nAR = compute_IWV_IVT_SST_pairs('nAR', region='ocean')

In [ ]:
k_IVT_AR, b_IVT_AR = compute_regr_coeffs(diff_SST_AR, diff_IVT_AR)
k_IVT_nAR, b_IVT_nAR = compute_regr_coeffs(diff_SST_nAR, diff_IVT_nAR)
k_IWV_AR, b_IWV_AR = compute_regr_coeffs(diff_SST_AR, diff_IWV_AR)
k_IWV_nAR, b_IWV_nAR = compute_regr_coeffs(diff_SST_nAR, diff_IWV_nAR)

## 3. plots

In [ ]:
fig1 = plt.figure(figsize=(7,7.5))
rowspan=9
colspan=10
ax1 = plt.subplot2grid((25,23), (0,0), colspan=colspan, rowspan=rowspan)
ax2 = plt.subplot2grid((25,23), (0,11), colspan=colspan, rowspan=rowspan)
ax3 = plt.subplot2grid((25,23), (12,0), colspan=colspan, rowspan=rowspan)
ax4 = plt.subplot2grid((25,23), (12,11), colspan=colspan, rowspan=rowspan)
axes_full = [ax1, ax2 ,ax3, ax4]



# ax1, moisture budget for AR
xmax = 25.5
ymax = 300
ax1.fill_between(hist_bin_ct, np.zeros(nbins), hist_AR, color='royalblue', alpha=0.5)
ax1.plot(np.ones(10)*ratio_AR.mean(), np.arange(0,1600,160), color='royalblue', lw=2)
#ax1.plot(np.ones(10)*m_ratio_AR, np.arange(0,1600,160), color='royalblue', lw=2, linestyle='--')
loc = ratio_AR.mean()
ax1.text(8, ymax*0.6, 'mean=%.1f%%' % loc, ha='left', va='center', fontsize=11)
#loc = m_ratio_AR
#ax1.text(8, ymax*0.45, 'moisture weighted\naverage=%.1f%%' % loc, ha='left', va='center', fontsize=11)
ax1.text(25.5*0.95, ymax*0.95, '(a) AR', ha='right', va='top', fontsize=13)
ax1.set_xlim([0, xmax])
ax1.set_ylim([0, ymax])


# ax2, moisture budget for non-AR
xmax = 60.5
ymax = 500
ax2.fill_between(hist_bin_ct, np.zeros(nbins), hist_nAR, color='royalblue', alpha=0.5)
ax2.plot(np.ones(10)*ratio_nAR.mean(), np.arange(0,1600,160), color='royalblue', lw=2)
#ax2.plot(np.ones(10)*m_ratio_nAR, np.arange(0,1600,160), color='royalblue', lw=2, linestyle='--')
loc = ratio_nAR.mean()
ax2.text(23, ymax*0.6, 'mean=%.1f%%' % loc, ha='left', va='center', fontsize=11)
#loc = m_ratio_nAR
#ax2.text(22, ymax*0.45, 'moisture weighted\naverage=%.1f%%' % loc, ha='left', va='center', fontsize=11)
ax2.text(xmax*0.95, ymax*0.95, '(b) non-AR', ha='right', va='top', fontsize=13)
ax2.set_xlim([0, xmax])
ax2.set_ylim([0, ymax])


# ax3 and ax4, sens
para_a = 0.08 # bin width
para_b = 0.05 # bin interval
colors4 = ['royalblue', 'lightseagreen']

xmax = 3
xres = 0.5
nbins2 = int(xmax*2/xres)
H_interval=xres
xdata = np.arange(-3, 6.1, 1)


barplot_double_set_sig(ax3, diff_SST_AR, [diff_IVT_AR, diff_IWV_AR], ['IVT', 'IWV'], ['royalblue', 'lightseagreen'])
barplot_double_set_sig(ax4, diff_SST_nAR, [diff_IVT_nAR, diff_IWV_nAR], ['IVT', 'IWV'], ['royalblue', 'lightseagreen'])
ax3.plot(xdata, xdata*k_IVT_AR+b_IVT_AR, color='royalblue')
ax3.plot(xdata, xdata*k_IWV_AR+b_IWV_AR, color='lightseagreen')
ax3.text(-0.2, 0.04*(-0.55), 'y=%.3fx+%.3f' % (k_IVT_AR, b_IVT_AR), color='royalblue', ha='left', va='bottom', fontsize=12)
ax3.text(-0.2, 0.04*(-0.9), 'y=%.3fx+%.3f' % (k_IWV_AR, b_IWV_AR), color='lightseagreen', ha='left', va='bottom', fontsize=12)
ax4.plot(xdata, xdata*k_IVT_nAR+b_IVT_nAR, color='royalblue')
ax4.plot(xdata, xdata*k_IWV_nAR+b_IWV_nAR, color='lightseagreen')
ax4.text(-0.2, 0.08*(-0.55), 'y=%.3fx+%.3f' % (k_IVT_nAR, b_IVT_nAR), color='royalblue', ha='left', va='bottom', fontsize=12)
ax4.text(-0.2, 0.08*(-0.9), 'y=%.3fx+%.3f' % (k_IWV_nAR, b_IWV_nAR), color='lightseagreen', ha='left', va='bottom', fontsize=12)


# decoration
for i in np.arange(nbins2):
    threb = -1*xmax+i*xres
    threu = -1*xmax+(i+1)*xres
    for axis in [ax3, ax4]:
        # background       
        if i%2==0:
            patch = matplotlib.patches.Rectangle((threb,-100), H_interval, 200, color='white', alpha=1, zorder=0)
        else:
            patch = matplotlib.patches.Rectangle((threb,-100), H_interval, 200, color='lightgray', alpha=0.6, zorder=0)
        axis.add_patch(patch)
         
for axis in [ax3, ax4]:
    axis.plot(np.arange(-20, 20), np.zeros(40), linestyle='--', linewidth=0.5, color='black')
    axis.set_xlim([-2, xmax])
    axis.set_xticks(np.arange(-2, xmax+0.000001, 1))
    axis.set_xlabel('SST perturbation (K)', fontsize=12)
    axis.set_xticklabels(np.arange(-2, xmax+xres+0.000001, 1, dtype=int))
    axis.set_ylabel('Relative change', fontsize=12)

ax3.set_ylim([-0.04, 0.08])
ax3.set_yticks(np.arange(-0.04, 0.08+0.000001, 0.04))
ax3.set_yticklabels(['-4%', '0', '+4%', '+8%'])

ax4.set_ylim([-0.08, 0.16])
ax4.set_yticks(np.arange(-0.08, 0.16+0.000001, 0.08))
ax4.set_yticklabels(['-8%', '0', '+8%', '+16%'])

for axis in [ax1, ax2]:
    axis.set_xlabel('E/(E+IVT)', fontsize=12)
    axis.set_ylabel('Frequency', fontsize=12)
    
ax1.set_xticks([0, 10, 20])
ax1.set_xticklabels(['0', '10%', '20%'], fontsize=12)
ax2.set_xticks([0, 20, 40, 60])
ax2.set_xticklabels(['0', '20%', '40%', '60%'], fontsize=12)

#ax1.text(28, 320, 'Contribution of ET in the total incoming moisture', ha='center', fontsize=14)
ax3.text(2.5, 0.075, '(c) AR', ha='right', va='top', fontsize=13)
ax4.text(2.5, 0.075*2, '(d) non-AR', ha='right', va='top', fontsize=13)

for axis in [ax2, ax4]:
    axis.yaxis.set_label_position("right")
    axis.yaxis.tick_right()

#fig1.savefig(plotdir + 'fig_S3.ET_moistuer_contrib.full_ocean.AR%s.png' % version_tag, dpi=600)

plt.show()
plt.close()
del(fig1)